In [1]:
import ee
ee.Initialize()

In [2]:
import ipygee as ui

In [3]:
from geetools import tools

In [4]:
import geepyGLAD as glad

In [5]:
Map = ui.Map()
Map.show()

Map(basemap={'max_zoom': 19, 'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'attribution': 'Map …

In [6]:
provincias = ee.FeatureCollection('users/rprincipe/norte_argentina')

In [7]:
def convert_provincias(feat):    
    n = ee.String(feat.get('NAM'))

    n = ee.String(ee.Algorithms.If(n.compareTo('Córdoba'), n, 'Cordoba'))
    n = ee.String(ee.Algorithms.If(n.compareTo('San Juan'), n, 'San_Juan'))
    n = ee.String(ee.Algorithms.If(n.compareTo('Santa Fe'), n, 'Santa_Fe'))
    n = ee.String(ee.Algorithms.If(n.compareTo('Santiago del Estero'), n, 'Santiago_del_Estero'))
    n = ee.String(ee.Algorithms.If(n.compareTo('Tucumán'), n, 'Tucuman'))

    return feat.set('NAME', ee.String(n))

In [8]:
provincias = provincias.map(convert_provincias)

In [9]:
def get_province(fc, field, name):
    return ee.Feature(fc.filterMetadata(field, 'equals', name).first())

In [10]:
tuc = get_province(provincias, 'NAME', 'Tucuman')

In [11]:
alert = glad.alerts.get_probable(tuc.geometry(), '2019-04-20')

In [12]:
Map.centerObject(tuc)

In [14]:
Map.addLayer(alert, name='probable')

In [18]:
confirmed = glad.alerts.get_confirmed(tuc.geometry(), '2019-04-20')

In [19]:
ui.eprint(glad.utils.histogram(confirmed, 'confirmed', tuc.geometry()))

In [20]:
Map.addLayer(confirmed, name='confirmed')

In [21]:
confpol = glad.utils.make_vector(confirmed, tuc.geometry())

In [24]:
confpol = confpol.map(lambda pol: pol.set('area', pol.geometry().area(1)))

In [25]:
Map.addLayer(confpol, name='confirmed vector')

In [15]:
filtered = glad.alerts.ALERTS.filterDate('2019-01-01', '2019-04-21').filterBounds(tuc.geometry())

In [16]:
mask = glad.alerts.last_probable_mask(filtered, 2019)

In [17]:
Map.addLayer(mask, name='probable mask')

In [14]:
alert2 = get_probable(tuc.geometry(), '2019-04-20')

In [32]:
Map.addLayer(alert2, name='alert local')

In [19]:
from geetools import collection

In [20]:
s2 = collection.Sentinel2TOA()

In [23]:
filt = s2.collection.filterMetadata(s2.cloud_cover, 'less_than', 10).filterBounds(tuc.geometry())

In [25]:
i = filt.first()

In [24]:
Map.addLayer(filt.first(), s2.visualization('NSR'), 'S2')

In [29]:
# Make the training dataset.
training = i.sample(**{
  'region': i.geometry(),
  'scale': 30,
  'numPixels': 5000
})

# Instantiate the clusterer and train it.
clusterer = ee.Clusterer.wekaKMeans(15).train(training)

# Cluster the input using the trained clusterer.
result = i.cluster(clusterer)

# Display the clusters with random colors

Map.addLayer(result.randomVisualizer(), dict(min=0, max=255), name='clusters')

In [ ]:
ee.FeatureCollection.getDownloadURL()